Import basic tools and Dance Object, used for turning joint-position data into a dataframe of features

In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append("../../")
from DanceProj1.DanceObj import Dance

Use Dance class obj on data and make feature-dataframe

In [2]:
from DanceProj1.data_proc import get_data

aistpath = '../../aist_keypoints' #path to 3d keypoints data
dataBM, dataFM = get_data(aistpath)    #get dataframes of features for Basic and Advanced dances

Make sure data is loaded

In [3]:
print('data is dict with keys = genres', dataBM.keys())
print('values are tuples (pairs) = (posdata, id)')
print('first break piece data shape (frames, joints, xyz):', dataBM['Break'][0][0].shape)
print('id of first break piece: ', dataBM['Break'][0][1])

data is dict with keys = genres dict_keys(['Break', 'Pop', 'Lock', 'Midhop', 'LAhop', 'House', 'Waack', 'Krump', 'Street Jazz', 'Ballet Jazz'])
values are tuples (pairs) = (posdata, id)
first break piece data shape (frames, joints, xyz): (15, 443, 3)
id of first break piece:  gBR_sBM_cAll_d05_mBR5_ch10


In [4]:
from DanceProj1.data_proc import data_to_features

dfBM, dfFM = data_to_features(dataBM, dataFM)

In [5]:
len(dfFM.index)

201

Check the dataframes

In [5]:
dfBM.head()

,Expandedness,Expandedness_std,Expandednessvel,Expandednessvel_std,Expandednessacc,Expandednessacc_std,Asym_RL_vel,Asym_RL_acc,Asym_RL_jer,Asym_RL_vel_std,...,Contracorr_Relb_Lknee_prominencej2,Contracorr_Relb_Lknee_prominencej3,Contracorr_Relb_Lknee_prominencej_std,Contracorr_Lelb_Rknee_heightjer,Contracorr_Lelb_Rknee_prominencej1,Contracorr_Lelb_Rknee_prominencej2,Contracorr_Lelb_Rknee_prominencej3,Contracorr_Lelb_Rknee_prominencej_std,id,Genre
0,1067.622148,50.415361,1397.204275,542.206065,7089.581203,2361.043942,-17940.791032,811.601870,-1632.939728,148.244525,...,5.596054e+08,4.063632e+08,9.199850e+07,1.587455e+08,3.348337e+08,3.047836e+08,2.417569e+08,3.878523e+07,gBR_sBM_cAll_d05_mBR5_ch10,Break
1,1083.575293,142.153402,1524.272988,333.127678,8006.105192,1837.480984,-233.891560,-1062.808853,321.412257,4.441060,...,6.038276e+08,5.554508e+08,1.250955e+08,1.300953e+08,8.625915e+08,6.590568e+08,6.263198e+08,1.045214e+08,gBR_sBM_cAll_d05_mBR1_ch03,Break
2,985.561049,104.975424,1907.621137,872.391745,8871.908429,2859.032792,-976.843869,-638.915156,-1542.022144,5.080327,...,4.701707e+08,3.745193e+08,6.341358e+07,4.185444e+08,8.902319e+08,6.626244e+08,6.419400e+08,1.124880e+08,gBR_sBM_cAll_d06_mBR3_ch09,Break
3,1020.270512,80.674363,2902.893592,874.548784,13713.203307,3345.844442,-439.064842,-49.327036,-307.920736,3.059820,...,7.964929e+08,5.529205e+08,2.676955e+08,3.259697e+08,7.382507e+08,5.326464e+08,2.906787e+08,1.829214e+08,gBR_sBM_cAll_d06_mBR2_ch08,Break
4,1107.946863,49.744991,1201.883329,435.443006,5814.821126,2057.817243,4432.851877,-357.711355,-392.961267,37.383511,...,2.086062e+08,1.969258e+08,4.661264e+07,9.989042e+07,2.320892e+08,1.750242e+08,1.681763e+08,2.865146e+07,gBR_sBM_cAll_d05_mBR4_ch10,Break


In [6]:
dfBM.columns

Index(['Expandedness', 'Expandedness_std', 'Expandednessvel',
       'Expandednessvel_std', 'Expandednessacc', 'Expandednessacc_std',
       'Asym_RL_vel', 'Asym_RL_acc', 'Asym_RL_jer', 'Asym_RL_vel_std',
       ...
       'Contracorr_Relb_Lknee_prominencej2',
       'Contracorr_Relb_Lknee_prominencej3',
       'Contracorr_Relb_Lknee_prominencej_std',
       'Contracorr_Lelb_Rknee_heightjer', 'Contracorr_Lelb_Rknee_prominencej1',
       'Contracorr_Lelb_Rknee_prominencej2',
       'Contracorr_Lelb_Rknee_prominencej3',
       'Contracorr_Lelb_Rknee_prominencej_std', 'id', 'Genre'],
      dtype='object', length=106)

In [7]:
dfFM.head()

,Expandedness,Expandedness_std,Expandednessvel,Expandednessvel_std,Expandednessacc,Expandednessacc_std,Asym_RL_vel,Asym_RL_acc,Asym_RL_jer,Asym_RL_vel_std,...,Contracorr_Relb_Lknee_prominencej2,Contracorr_Relb_Lknee_prominencej3,Contracorr_Relb_Lknee_prominencej_std,Contracorr_Lelb_Rknee_heightjer,Contracorr_Lelb_Rknee_prominencej1,Contracorr_Lelb_Rknee_prominencej2,Contracorr_Lelb_Rknee_prominencej3,Contracorr_Lelb_Rknee_prominencej_std,id,Genre
0,1056.865643,90.082477,1538.228504,958.219385,7666.414735,3900.555376,2839.095566,3202.027147,-8145.973932,6.328799,...,1.664854e+09,1.434043e+09,1.443755e+08,7.402209e+07,1.229382e+09,1.192352e+09,1.179422e+09,2.117252e+07,gBR_sFM_cAll_d05_mBR2_ch09,Break
1,1100.526992,108.640735,1619.546822,761.437478,8351.133624,3748.230439,1330.346396,-3083.023419,-94451.348976,15.941952,...,1.614114e+09,1.565013e+09,1.055785e+08,3.317929e+08,1.771329e+09,1.737268e+09,1.393618e+09,1.705940e+08,gBR_sFM_cAll_d04_mBR5_ch06,Break
2,1012.354348,101.102753,1380.734980,483.503378,6921.930683,2232.699927,2425.553893,-3654.077082,2469.866960,19.227707,...,1.208936e+09,1.112264e+09,1.159281e+08,3.778749e+08,1.274994e+09,9.255891e+08,9.058842e+08,1.695464e+08,gBR_sFM_cAll_d06_mBR2_ch16,Break
3,1081.452194,108.619361,1552.177055,723.052643,7303.880780,3075.843913,2195.992765,2114.188199,-759.887878,8.031298,...,8.673836e+08,7.578112e+08,5.950155e+07,1.357860e+08,6.830069e+08,5.732215e+08,5.102448e+08,7.138754e+07,gBR_sFM_cAll_d04_mBR3_ch04,Break
4,1089.897141,676.195640,2856.621415,3074.723430,14871.032459,15856.841540,-6285.452239,233.536330,-2404.161128,43.765981,...,1.745922e+09,1.598506e+09,1.979416e+08,4.219170e+09,7.735069e+09,6.359036e+09,5.894639e+09,7.814742e+08,gBR_sFM_cAll_d06_mBR4_ch20,Break
